In [2]:
from operator import mul
from scipy.special import binom
from functools import reduce
from itertools import product
from math import factorial

In [3]:
def num_partitions(k, n):
    if k == 0 and n == 0:
        return 1
    if n <= 0 or k <= 0:
        return 0
    return num_partitions(k, n-k) + num_partitions(k-1, n-1)

In [4]:
def partitions(n, max_length):
    L = [[1]]
    for _ in range(n-1):
        L2 = []
        for l in L:
            L2.append(l[:-1] + [l[-1] + 1])
            if len(l) < max_length:
                L2.append(l + [1])
        L = L2
    return [l for l in L if len(l) > 1]

In [5]:
def fixed_length_partitions(n, num_partitions, min_value=1):
    assert n >= num_partitions
    L = [[]]
    for i in range(num_partitions - 1):
        L2 = []
        for l in L:
            L2 += [l + [k] for k in range(min_value, n - sum(l) + 1 - (num_partitions - 1 - i))]
        L = L2
    for l in L:
        l.append(n - sum(l))
    return L

In [6]:
def ordered_subsets(n, max_length):
    L = [[]]
    while True:
        small_L = [l for l in L if len(l) < max_length]
        if len(small_L) == 0:
            break
        L = []
        for l in small_L:
            for i in range(l[-1]+1 if len(l) > 0 else 1, n+1):
                yield l + [i]
                L.append(l + [i])

In [7]:
def count_programs(arities, predicates_with_arity, num_variables, num_constants, max_num_nodes, max_num_clauses):
    num_predicates = sum(predicates_with_arity)
    def arity(a):
        'The arity of predicate indexed at a'
        i = 0
        while (a + 1 > predicates_with_arity[i]):
            a -= predicates_with_arity[i]
            i += 1
        return arities[i]

    def P(n):
        t = num_constants**n
        for s in ordered_subsets(n, num_variables):
            s = [0] + s + [n+1]
            t += reduce(mul, [(num_constants + i)**(s[i+1] - s[i] - 1) for i in range(len(s) - 1)], 1)
        #print('P(' + str(n) + ') =', t)
        return t

    def T(n, a):
        if n == 1:
            return predicates_with_arity[arities.index(a)] if a in arities else 0
        s = 0
        for partition1 in partitions(n-1, a / min(arities) if min(arities) > 0 else float('inf')):
            for partition2 in fixed_length_partitions(a, len(partition1), min(arities)):
                s += reduce(mul, [T(k, l) for k, l in zip(partition1, partition2)], 1)
        return T(n-1, a) + 2 * s

    def C(a):
        if a == 0:
            return 1
        return sum(T(n, a) for n in range(1, max_num_nodes + 1))

    s = 0
    for n in range(num_predicates, max_num_clauses + 1):
        for partition in fixed_length_partitions(n, num_predicates):
            m = 1
            for i,h in enumerate(partition):
                t = 0
                for a in range(max(arities) * max_num_nodes + 1):
                    foo = int(C(a) * P(a + arity(i)))
                    #print('arity', a, 'gets', foo, 'possibilities')
                    t += foo
                m *= int(binom(t + h - 1, h))
            #print(partition, m)
            s += m
    return s

In [8]:
arities = [[1], [2], [3], [2, 1], [4], [3, 1]]
r = list(range(1, 5))
predicates_with_arity = {1: r, 2: list(product(r, r)), 3: list(product(r, r, r)), 4: list(product(r, r, r, r))}
num_variables = r
num_constants = range(0, 4)
max_num_nodes = r
MAX = 100000

f = open('program_counts.csv', 'w+')
for arity in arities:
    for pred in predicates_with_arity[len(arity)]:
        if isinstance(pred, tuple):
            pred = list(pred)
        elif not isinstance(pred, list):
            pred = [pred]
        num_pred = sum(pred)
        for num_var in num_variables:
            for num_const in num_constants:
                for max_nodes in max_num_nodes:
                    for max_clauses in range(num_pred, num_pred + 6):
                        count = count_programs(arity, pred, num_var, num_const, max_nodes, max_clauses)
                        if count > MAX:
                            break
                        d = [arity, pred, num_var, num_const, max_nodes, max_clauses, count]
                        s = ';'.join([str(t) for t in d])
                        f.write(s+'\n')
f.close()

In [ ]:
count_programs([1], [1], 1, 1, 1, 2)

In [ ]:
num_partitions(3, 2)

In [ ]:
list(ordered_subset(3, 2))